# Create Vertex AI Search

In [ ]:
import pandas as pd
import json

import requests
import subprocess

project_id = "rl-llm-dev"

In [ ]:
def get_access_token_from_metadata():
    # Retrieve the access token using the gcloud command
    access_token_command = subprocess.run(['gcloud', 'auth', 'print-access-token'], capture_output=True)
    token = access_token_command.stdout.decode('utf-8').strip()
    return token


def post_request_vertex_search(data, url, headers):
    # Send the POST request
    response = requests.post(url, headers=headers, json=data)
    print(response.text)

    # Handle the response (Optional: based on your requirement)
    if response.status_code == 200:
        print("Request was successful!")
    else:
        print("Failed with status code:", response.status_code)


def patch_request_vertex_search(data, url, headers):
    # Send the POST request
    response = requests.patch(url, headers=headers, json=data)
    print(response.text)

    # Handle the response (Optional: based on your requirement)
    if response.status_code == 200:
        print("Request was successful!")
    else:
        print("Failed with status code:", response.status_code)

In [ ]:
# Create Datastore with Alpha features
# Set the URL
datastore_id = "csm-website-ds"
url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/global/collections/default_collection/dataStores?dataStoreId={datastore_id}"

# Set the payload/data
data = {
    "displayName": "csm-website-ds",
    "industryVertical": "GENERIC",
    "solutionTypes": ["SOLUTION_TYPE_SEARCH"],
    "searchTier": "ENTERPRISE",
    "searchAddOns": ["LLM"]
}

# Set the headers
headers = {
    "Authorization": f"Bearer {get_access_token_from_metadata()}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": f"{project_id}"
}

post_request_vertex_search(data, url, headers)

In [ ]:
# Set the headers
headers = {
    "Authorization": f"Bearer {get_access_token_from_metadata()}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": f"{project_id}"
}

# Set the payload/data
with open("./schema.json", "r") as f:
    schema = json.load(f)

request_body = {
    "structSchema": schema
}

request_url = "https://discoveryengine.googleapis.com/v1beta/projects/rl-llm-dev/locations/global/collections/default_collection/dataStores/csm-website-ds/schemas/default_schema"

response = requests.patch(request_url, headers=headers, json=request_body)

if response.status_code == 200:
    print('Schema updated successfully')
else:
    print(f'Error updating schema: {response.status_code}')

### Upload items to search

In [ ]:
input_file = pd.read_excel("./csm-dataset.xlsx")

# Set the headers
headers = {
    "Authorization": f"Bearer {get_access_token_from_metadata()}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": f"{project_id}"
}

url = "https://discoveryengine.googleapis.com/v1beta/projects/rl-llm-dev/locations/global/collections/default_collection/dataStores/csm-website-ds/branches/0/documents?documentId={document_id}"

In [ ]:
for item in input_file.iterrows():
    data = {
        "structData": item[1].to_dict()
    }
    post_request_vertex_search(data=data, url=url.format(document_id=item[1]["id"]), headers=headers)

### Search App

In [ ]:
from google.cloud import discoveryengine_v1beta as discoveryengine

In [ ]:
search_client = discoveryengine.SearchServiceClient()
project_id = "rl-llm-dev"

In [ ]:
serving_config = search_client.serving_config_path(
    project=project_id,
    location="global",
    data_store="csm-website-ds",
    serving_config="default_config",
)

request = discoveryengine.SearchRequest(
    serving_config=serving_config,
    query="are gaming chairs ergonomic?"
)

In [ ]:
response = search_client.search(request)
response

In [ ]:
response.results

In [ ]:
datastore_id = "csm-pdfs-test_1696763354868"

url = f"https://discoveryengine.googleapis.com/v1beta/projects/{project_id}/locations/global/collections/default_collection/dataStores/{datastore_id}/servingConfigs/default_search:search"

data = {
    "servingConfig": f"projects/{project_id}/locations/global/collections/default_collection/dataStores/{datastore_id}/servingConfigs/default_search",
    "query": "nylon chair",
    "pageSize": "100",
    "offset": "0",
    "contentSearchSpec": {
        "summarySpec": {
            "summaryResultCount": 3,
            "includeCitations": True
        },
        "extractiveContentSpec": { 
            "maxExtractiveAnswerCount" : 1
        },
    }
}

# Set the headers
headers = {
    "Authorization": f"Bearer {get_access_token_from_metadata()}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": f"{project_id}"
}

In [ ]:
post_request_vertex_search(
    data, url, headers
)

In [ ]:
datastore_id = "csm-pdfs-test_1696763354868"
project_id = "rl-llm-dev"

url = f"https://discoveryengine.googleapis.com/v1beta/projects/{project_id}/locations/global/collections/default_collection/dataStores/{datastore_id}/servingConfigs/default_search:search"

data = {
    "servingConfig": f"projects/{project_id}/locations/global/collections/default_collection/dataStores/{datastore_id}/servingConfigs/default_search",
    "pageSize": "100",
    "offset": "0",
    "contentSearchSpec": {
        "summarySpec": {
            "summaryResultCount": 3,
            "includeCitations": True
        },
        "extractiveContentSpec": { 
            "maxExtractiveAnswerCount" : 1
        },
    },
    "query": "Are gaming chairs confortable?",
    "params": {"search_type": 0}
}

# Set the headers
headers = {
    "Authorization": f"Bearer {get_access_token_from_metadata()}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": f"{project_id}"
}

In [ ]:
post_request_vertex_search(
    data, url, headers
)

In [ ]:
from google.cloud import discoveryengine_v1beta as discoveryengine

In [ ]:
search_client = discoveryengine.SearchServiceClient()
project_id = "rl-llm-dev"
search_location = "global"
datastore = "csm-pdfs-test_1696763354868"
serving_config = "default_config"

In [ ]:
serving_config = search_client.serving_config_path(
    project=project_id,
    location=search_location,
    data_store=datastore,
    serving_config=serving_config
)

snippet_spec = discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
    return_snippet=True)
summary_spec = discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
    summary_result_count=1, include_citations=True)
extractive_content_spec = discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
    max_extractive_answer_count=0,
    max_extractive_segment_count=0,
    return_extractive_segment_score=False,
    num_previous_segments=0,
    num_next_segments=0
)

content_spec = discoveryengine.SearchRequest.ContentSearchSpec(
    snippet_spec=snippet_spec,
    summary_spec=summary_spec,
    extractive_content_spec=extractive_content_spec
)

request = discoveryengine.SearchRequest(
    content_search_spec=content_spec,
    serving_config=serving_config,
    query="are gaming chairs ergonomic?"
)

In [ ]:
response = search_client.search(request)
response

In [ ]:
from google.cloud import discoveryengine_v1beta as discoveryengine
search_client = discoveryengine.SearchServiceClient()
project_id = "rl-llm-dev"
search_location = "global"
datastore = "csm-pdfs-test_1696763354868"
serving_config = "default_config"

In [ ]:
import utils_search

In [ ]:
test = utils_search.search_website_text(
    search_client=search_client,
    search_query="Are gamer chairs ergonimic?",
    datastore_location=search_location,
    project_id="rl-llm-dev",
    datastore_id=datastore,
)

### Follow up

In [1]:
from google.cloud import discoveryengine_v1beta as discoveryengine

In [2]:
conversational_client = discoveryengine.ConversationalSearchServiceClient()
project_id = "rl-llm-dev"
search_location = "global"
datastore = "csm-products_1696914795319"
serving_config_id = "default_config"

In [29]:
converse_request = discoveryengine.CreateConversationRequest(
    parent = f"projects/{project_id}/locations/{search_location}/collections/default_collection/dataStores/{datastore}",
    conversation=discoveryengine.Conversation(
        user_pseudo_id="renato2"
    )
)

In [30]:
conversation = conversational_client.create_conversation(
    request=converse_request
)

In [31]:
conversation

name: "projects/244831775715/locations/global/collections/default_collection/dataStores/csm-products_1696914795319/conversations/4241441740720356736"
state: IN_PROGRESS
user_pseudo_id: "renato2"
start_time {
  seconds: 1697828934
  nanos: 935471000
}

In [35]:
serving_config = conversational_client.serving_config_path(
    project=project_id,
    location=search_location,
    data_store=datastore,
    serving_config=serving_config_id
)

summary_spec = discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(include_citations=True)

conversation_request = discoveryengine.ConverseConversationRequest(
    name=conversation.name,
    query=discoveryengine.TextInput(input="Are gaming chairs ergonomic?"),
    serving_config=serving_config,
    summary_spec=summary_spec
)

In [36]:
request = conversational_client.converse_conversation(
    request=conversation_request
)

In [37]:
request

reply {
  reply: "Yes, some gaming chairs are ergonomically designed to support your back and spine [1, 2]. They can help you develop a healthy sitting posture and relieve back pain [1]."
  summary {
    summary_text: "Yes, some gaming chairs are ergonomically designed to support your back and spine [1, 2]. They can help you develop a healthy sitting posture and relieve back pain [1]."
    safety_attributes {
      categories: "Health"
      categories: "Insult"
      categories: "Toxic"
      scores: 0.4
      scores: 0.1
      scores: 0.1
    }
  }
}
conversation {
  name: "projects/244831775715/locations/global/collections/default_collection/dataStores/csm-products_1696914795319/conversations/4241441740720356736"
  state: IN_PROGRESS
  user_pseudo_id: "renato2"
  messages {
    user_input {
      input: "Are gaming chairs ergonomic?"
    }
  }
  messages {
    reply {
      reply: "Some gaming chairs are ergonomically designed to support your back and spine [1, 2]. They can help you

In [ ]:
type(request)

In [10]:
conversation_request = discoveryengine.ConverseConversationRequest(
    name=f"projects/{project_id}/locations/{search_location}/collections/default_collection/dataStores/{datastore}/conversations/-",
    query=discoveryengine.TextInput(input="Nylon chair too?"),
    serving_config=serving_config,
    conversation=conversation,
    summary_spec=summary_spec
)

request = conversational_client.converse_conversation(
    request=conversation_request
)

In [24]:
from proto import Message

In [27]:
search_result_dict = Message.to_dict(request.search_results[0])

In [28]:
document = search_result_dict.get("document", {})

In [36]:
document["derived_struct_data"]["extractive_answers"]
# document["derived_struct_data"]["snippets"]

[{'content': '【Durable Performance】:The gaming chair features an all-steel frame for superior weight-bearing performance. The chair is equipped with a nylon base and nylon casters that roll smoothly on the floor without noise, providing excellent mobility.',
  'pageNumber': '1'}]

In [40]:
conv = Message.to_dict(request.conversation)

In [63]:
conv["messages"][1]["reply"]["reply"]

'Yes, gaming chairs are ergonomically designed to support your back and spine [1]. They typically have a wide, curved back that provides support for your lumbar region, and they also have adjustable armrests and headrests to help you find a comfortable position.'

In [65]:
reply = Message.to_dict(request.reply)

In [67]:
reply["reply"]

'Yes, nylon chairs are also ergonomically designed to support your back and spine [2]. They typically have a wide, curved back that provides support for your lumbar region, and they also have adjustable armrests and headrests to help you find a comfortable position.'

In [31]:
request

reply {
  reply: "Yes, nylon chairs are also ergonomically designed to support your back and spine [2]. They typically have a wide, curved back that provides support for your lumbar region, and they also have adjustable armrests and headrests to help you find a comfortable position."
  summary {
    summary_text: "Yes, nylon chairs are also ergonomically designed to support your back and spine [2]. They typically have a wide, curved back that provides support for your lumbar region, and they also have adjustable armrests and headrests to help you find a comfortable position."
    safety_attributes {
      categories: "Health"
      categories: "Insult"
      categories: "Toxic"
      scores: 0.4
      scores: 0.1
      scores: 0.1
    }
  }
}
conversation {
  name: "projects/244831775715/locations/global/collections/default_collection/dataStores/csm-pdfs-test_1696763354868/conversations/3719273491569173825"
  state: IN_PROGRESS
  user_pseudo_id: "renato"
  messages {
    user_input {
 

In [22]:
request.search_results

[id: "0a126579e1f454658db0124fd4ebcf15"
document {
  name: "projects/244831775715/locations/global/collections/default_collection/dataStores/csm-pdfs-test_1696763354868/branches/0/documents/0a126579e1f454658db0124fd4ebcf15"
  id: "0a126579e1f454658db0124fd4ebcf15"
  derived_struct_data {
    fields {
      key: "snippets"
      value {
        list_value {
          values {
            struct_value {
              fields {
                key: "snippet"
                value {
                  string_value: "Title: JOYFLY Gaming <b>Chairs</b> with Footrest, <b>Ergonomic</b> High Back Gaming <b>Chair</b> for Adults Teens, Reclining Computer <b>Chair</b> with Headrest &amp; Lumbar Support,350lbs&nbsp;..."
                }
              }
              fields {
                key: "snippet_status"
                value {
                  string_value: "SUCCESS"
                }
              }
            }
          }
        }
      }
    }
    fields {
      key: "link"
      val

# Image

In [ ]:
import utils_search

In [ ]:
from google.cloud import discoveryengine_v1beta as discoveryengine
search_client = discoveryengine.SearchServiceClient()
project_id = "rl-llm-dev"
search_location = "global"
datastore = "csm-website-ds"
serving_config = "default_config"

In [ ]:
utils_search.search_website_image(
    search_client=search_client,
    project_id=project_id,
    datastore_location=search_location,
    datastore_id=datastore,
    image_bytes=b''
)

In [68]:
from google.cloud import firestore

db = firestore.Client()

In [69]:
test = db.collection("test2").document().set({"nome": "renato"})

In [ ]:
test.set({"nome": "renato"})

In [70]:
test.id

AttributeError: Unknown field for WriteResult: id